<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_diffusers_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install --upgrade diffusers accelerate transformers xformers safetensors
!mkdir -p outputs/{txt2img,img2img}

In [ ]:
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git
!git clone https://huggingface.co/embed/negative /content/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/Lora/positive

In [ ]:
import torch
from diffusers import DiffusionPipeline

output_index = 0
generator = torch.Generator()

# Default settings
save_images = True
display_images = True

# Creating model pipeline
You can use most from [huggingface](https://huggingface.co). Just make sure they are in diffusers format, check their **Files** and see if there is a `model_index.json`.

I made a simple index wih popular models, so you can just render UI and choose a model from dropdown. Some models require trigger word in the prompt.

For weights in Automatic111 format (**.ckpt** or **.safetensors** but without model_index.json), you'll need to download them and convert using scripts (WIP).

In [ ]:
#@title Render model choice UI
import json
import ipywidgets as widgets

with open("/content/stable-diffusion-diffusers-colab-ui/model_index.json") as f:
    data = json.load(f)

model_dropdown = widgets.Dropdown(
    options=[x for x in data],
    description="Model:",
)
model_link = widgets.HTML()

def set_label_from_dict(key):
    model_link.value = f"Model info: <a href=https://huggingface.co/{data[key]['id']}>{key}</a>"
    try:
        model_link.value += f"<br>Trigger prompt: <code>{data[key]['trigger']}</code>"
    except: pass

def dropdown_eventhandler(change):
    set_label_from_dict(change.new)

set_label_from_dict(model_dropdown.value)
model_dropdown.observe(dropdown_eventhandler, names='value')

display(model_dropdown, model_link)

In [ ]:
#@title Load chosen model
#@markdown Alternatively you can just paste huggingface model_id or path to model folder here:

path_to_model = ""  #@param {type: "string"}

if path_to_model != "": model_id = path_to_model
else: model_id = data[model_dropdown.value]['id']

pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = None

## VAE

Models loaded from huggingface should have proper VAE. But you also can load VAE from any other repository from [huggingface](https://huggingface.co), just paste their **model_id** and subfolder (usually just "vae"). 

Or you can load it in Automatic111 format and convert using scripts.

In [ ]:
#@title (Optional) Load VAE

from diffusers import AutoencoderKL

vae_id_or_path = "runwayml/stable-diffusion-v1-5"  #@param {type: "string"}
vae_subfolder = "vae"    #@param {type: "string"}

vae = AutoencoderKL.from_pretrained("vae_id_or_path", subfolder=vae_subfolder, torch_dtype=torch.float16).to("cuda")
pipe.vae = vae

## Textual inversions and LoRAs
This is work in progress. 

You can load any textual inversions using 
```
pipe.load_textual_inversion("path/to/dir", weight_name="filename.pt")
```
To use them, you need to add theit token to a prompt like this `<filename>`.

Note, that it seems that if you change VAE after loading textual inversions, using them will degrade generated images.

In [ ]:
#@title (Optional) Load textual inversions
#@markdown Load every embedding with **.pt** extension from **embeddings** folder.

import os

for path, subdirs, files in os.walk("/content/embeddings/"):
    for name in files:
        if os.path.splitext(name)[1] != ".pt": continue
        pipe.load_textual_inversion(path, weight_name=name)
        print(path, name)

# Generating images

In [ ]:
#@title Settings
#@markdown Run this cell to update
save_images = True #@param {type:"boolean"}
display_images = True    #@param {type:"boolean"}

In [ ]:
#@title Run this cell to generate images
prompt = "best quality, 1girl, illustration"  #@param {type: "string"}
negative_prompt = "<EasyNegative>, by <bad-artist-anime>, worst quality, awful quality"   #@param {type: "string"}

width = 512     #@param {type:"integer"}
height = 768    #@param {type:"integer"}
#@markdown ---

steps = 20  #@param {type: "slider", min: 1, max: 100}
CFG = 7     #@param {type: "slider", min: 1, max: 20}
seed = -1   #@param {type:"integer"}
#@markdown ---
batch_size = 1   #@param {type:"integer"}
#@markdown ---

if seed >= 0: generator = torch.manual_seed(seed)

results = pipe([prompt]*batch_size, negative_prompt=[negative_prompt]*batch_size, 
               num_inference_steps=steps, guidance_scale=CFG, 
               generator=generator, height=height, width=width)

# Save and display
for image in results.images:
    if save_images:
        image.save(f"outputs/txt2img/{output_index:05}.png")
        print(f"outputs/txt2img/{output_index:05}.png")
        output_index += 1
    if display_images: 
        display(image)
